In [1]:
from keras.layers import Input, Dense
from keras.models import Model
import numpy as np
import pandas as pd
import tqdm

Using TensorFlow backend.


In [26]:
# this is the size of our encoded representations
encoding_dim = 300  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input = Input(shape=(1000,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(input)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(1000, activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input, decoded)

In [27]:
# this model maps an input to its encoded representation
encoder = Model(input, encoded)

In [28]:
# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

In [29]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [30]:
data = np.load('data/body2vec.npy')
x_train = data[:100000,:]
x_test = data[-(data.shape[0]-x_train.shape[0]):,:]

print data.shape
print x_train.shape
print x_test.shape

(128972, 1000)
(100000, 1000)
(28972, 1000)


In [31]:
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

Train on 100000 samples, validate on 28972 samples
Epoch 1/50
100000/100000 [==============================] - 6s - loss: 0.4822 - val_loss: 0.0852
Epoch 2/50
100000/100000 [==============================] - 3s - loss: 0.0365 - val_loss: 0.0158
Epoch 3/50
100000/100000 [==============================] - 3s - loss: 0.0152 - val_loss: 0.0105
Epoch 4/50
100000/100000 [==============================] - 3s - loss: 0.0122 - val_loss: 0.0090
Epoch 5/50
100000/100000 [==============================] - 3s - loss: 0.0112 - val_loss: 0.0083
Epoch 6/50
100000/100000 [==============================] - 3s - loss: 0.0108 - val_loss: 0.0080
Epoch 7/50
100000/100000 [==============================] - 3s - loss: 0.0105 - val_loss: 0.0078
Epoch 8/50
100000/100000 [==============================] - 3s - loss: 0.0103 - val_loss: 0.0076
Epoch 9/50
100000/100000 [==============================] - 3s - loss: 0.0102 - val_loss: 0.0075
Epoch 10/50
100000/100000 [==============================] - 3s - loss: 0.01

In [38]:
encoded = encoder.predict(data)
encoded.shape

(128972, 300)

In [39]:
np.save('data/body2vec_autoencoder',encoded)

In [40]:
np.load('data/body2vec_autoencoder.npy').shape

(128972, 300)